In [2]:
import json

import pprint
pp = pprint.PrettyPrinter(indent=4)

from lxml import etree
from analysis.xml import DocumentInfo#, ElementInfo

#### Extract Ableton XML file into a DocumentInfo structure

In [3]:
tree = etree.parse('eda.als.xml')
root = tree.getroot()
doc = DocumentInfo(root)

#### Get all unique element tag names

In [4]:
all_element_tag_names = doc.get_all_element_tag_names()
pp.pprint(all_element_tag_names)

{   'Ableton',
    'Active',
    'AllPassGain',
    'AllPassSize',
    'AnchorTime',
    'Annotation',
    'ArrangementOverdub',
    'ArrangerAutomation',
    'ArrangerIO',
    'ArrangerMixer',
    'ArrangerReturns',
    'ArrangerShowOverView',
    'ArrangerTrackDelay',
    'AudioInputRouting',
    'AudioOutputRouting',
    'AudioSequencer',
    'AudioTrack',
    'AutoColorPickerForPlayerAndGroupTracks',
    'AutoColorPickerForReturnAndMasterTracks',
    'AutoQuantisation',
    'Automation',
    'AutomationEnvelope',
    'AutomationEnvelopes',
    'AutomationLane',
    'AutomationLanes',
    'AutomationMode',
    'AutomationTarget',
    'BandFreq',
    'BandHighOn',
    'BandLowOn',
    'BandWidth',
    'BeatDelayEnumL',
    'BeatDelayEnumR',
    'BeatTimeHelper',
    'BranchDeviceId',
    'BranchSourceContext',
    'BrowserContentPath',
    'ChooserBar',
    'ChorusOn',
    'ClientSize',
    'ClipEnvelopeChooserViewState',
    'ClipSlot',
    'ClipSlotList',
    'ClipSlotsListWrapper'

#### Get all unique element attribute names

In [5]:
all_attribute_names = doc.get_all_attribute_names()
pp.pprint(all_attribute_names)

{   'Bottom',
    'Creator',
    'Dir',
    'Id',
    'Left',
    'LomId',
    'MajorVersion',
    'MinorVersion',
    'Revision',
    'Right',
    'SchemaChangeCount',
    'Time',
    'Top',
    'Value',
    'X',
    'Y'}


#### Group together all elements that share an identically valued attribute

In [6]:
for attribute_name in all_attribute_names:
    print("----------------------------------------------------")
    print(f"{attribute_name}")
    print("All the attribute's unique values and the corresponding elements containing them")
    print("----------------------------------------------------")
    elements_info_grouped_by_id = doc.get_elements_info_grouped_by_attribute(attribute_name, show_only_unique_tag_names=True)
    pp.pprint(elements_info_grouped_by_id)

----------------------------------------------------
LomId
All the attribute's unique values and the corresponding elements containing them
----------------------------------------------------
{   '0': [   'DevicesListWrapper',
             'CuePointsListWrapper',
             'ClipSlotsListWrapper',
             'TracksListWrapper',
             'ScenesListWrapper',
             'SendsListWrapper',
             'VisibleTracksListWrapper',
             'ParametersListWrapper',
             'ReturnTracksListWrapper']}
----------------------------------------------------
Time
All the attribute's unique values and the corresponding elements containing them
----------------------------------------------------
{'-63072000': ['FloatEvent', 'EnumEvent']}
----------------------------------------------------
Id
All the attribute's unique values and the corresponding elements containing them
----------------------------------------------------
{   '0': [   'ClipSlot',
             'FilePresetRef